# Advent of Code 2021

Import libraries and utilities.

In [1]:
from __future__  import annotations
from collections import Counter, defaultdict, namedtuple, deque
from itertools   import permutations, combinations, chain, count as count_from, product as cross_product
from typing      import *
from statistics  import mean, median
from math        import ceil, inf
from functools   import lru_cache
import matplotlib.pyplot as plt
import re

def answer(puzzle_number, got, expected) -> bool:
    """Verify the answer we got was the expected answer."""
    assert got == expected, f'For {puzzle_number}, expected {expected} but got {got}.'
    return True

def parse(day, parser=str, sep='\n', print_lines=7) -> tuple:
    """Split the day's input file into entries separated by `sep`, and apply `parser` to each."""
    fname = f'input{day}.txt' # Suited to my needs
    text  = open(fname).read()
    entries = mapt(parser, text.rstrip().split(sep))
    if print_lines:
        all_lines = text.splitlines()
        lines = all_lines[:print_lines]
        head = f'{fname} ➜ {len(text)} chars, {len(all_lines)} lines; first {len(lines)} lines:'
        dash = "-" * 100
        print(f'{dash}\n{head}\n{dash}')
        for line in lines:
            print(trunc(line))
        print(f'{dash}\nparse({day}) ➜ {len(entries)} entries:\n'
              f'{dash}\n{trunc(str(entries))}\n{dash}')
    return entries

def trunc(s: str, left=70, right=25, dots=' ... ') -> str: 
    """All of string s if it fits; else left and right ends of s with dots in the middle."""
#     dots = ' ... ' No need
    return s if len(s) <= left + right + len(dots) else s[:left] + dots + s[-right:]

Char = str # Intended as the type of a one-character string
Atom = Union[float, int, str]

def ints(text: str) -> Tuple[int]:
    """A tuple of all the integers in text, ignoring non-number characters."""
    return mapt(int, re.findall(r'-?[0-9]+', text))

def digits(text: str) -> Tuple[int]:
    """A tuple of all the digits in text (as ints 0–9), ignoring non-digit characters."""
    return mapt(int, re.findall(r'[0-9]', text))

def words(text: str) -> List[str]:
    """A list of all the alphabetic words in text, ignoring non-letters."""
    return re.findall(r'[a-zA-Z]+', text)

def atoms(text: str) -> Tuple[Atom]:
    """A tuple of all the atoms (numbers or symbol names) in text."""
    return mapt(atom, re.findall(r'[a-zA-Z_0-9.+-]+', text))

def atom(text: str) -> Atom:
    """Parse text into a single float or int or str."""
    try:
        x = float(text)
        return round(x) if round(x) == x else x
    except ValueError:
        return text
    
def mapt(fn, *args) -> tuple:
    """map(fn, *args) and return the result as a tuple."""
    return tuple(map(fn, *args))

def quantify(iterable, pred=bool) -> int:
    """Count the number of items in iterable for which pred is true."""
    return sum(1 for item in iterable if pred(item))

class multimap(defaultdict):
    """A mapping of {key: [val1, val2, ...]}."""
    def __init__(self, pairs: Iterable[tuple], symmetric=False):
        """Given (key, val) pairs, return {key: [val, ...], ...}.
        If `symmetric` is True, treat (key, val) as (key, val) plus (val, key)."""
        self.default_factory = list
        for (key, val) in pairs:
            self[key].append(val)
            if symmetric:
                self[val].append(key)

def prod(numbers) -> float: # Will be math.prod in Python 3.8
    """The product formed by multiplying `numbers` together."""
    result = 1
    for x in numbers:
        result *= x
    return result

def total(counter: Counter) -> int: 
    """The sum of all the counts in a Counter."""
    return sum(counter.values())

def sign(x) -> int: return (0 if x == 0 else +1 if x > 0 else -1)

def transpose(matrix) -> list: return list(zip(*matrix))

def nothing(*args) -> None: return None

cat     = ''.join
flatten = chain.from_iterable
cache   = lru_cache(None)

## Day 1

Problem: How many times the value increased from the previous one?

In [8]:
inp1 = parse(1, parser=int)

----------------------------------------------------------------------------------------------------
input1.txt ➜ 9777 chars, 2000 lines; first 7 lines:
----------------------------------------------------------------------------------------------------
151
152
153
158
159
163
164
----------------------------------------------------------------------------------------------------
parse(1) ➜ 2000 entries:
----------------------------------------------------------------------------------------------------
(151, 152, 153, 158, 159, 163, 164, 162, 161, 167, 169, 168, 169, 170, ... , 8078, 8081, 8112, 8127)
----------------------------------------------------------------------------------------------------


In [51]:
# A sample provided by Advent of Code
sample1 = [199, 200, 208, 210, 200, 207, 240, 269, 260, 263]

In [35]:
# My naive attempt to solve
def deeper(inputs):
    return sum(1 for x, y in zip(inputs[:-1], inputs[1:]) if y > x)

In [94]:
# Way better version with quantify
def deeper_quant(inputs):
    return quantify(inputs[i + 1] > inputs[i] for i in range(len(inputs) - 1))

In [92]:
deeper(sample1)

7

In [95]:
answer('sample1', deeper_quant(sample1), 7)

True

In [37]:
deeper(inp1)

1564

In [96]:
answer(1.1, deeper_quant(inp1), 1564)

True

Problem 2: Three-measurement sliding window. Instead of comparing one item to another, we now compare a tripplet to next tripplet.

In [54]:
# My naive version
def deeper2(inputs):
    three_measurements = [x+y+z for x, y, z in 
                          zip(inputs[:-2], inputs[1:-1], inputs[2:])]
    return sum(1 for x, y in zip(three_measurements[:-1], three_measurements[1:]) if y > x)

In [86]:
# Better version with quantify
def deeper_quant2(inputs):
    return quantify(sum(inputs[i:i+3]) < sum(inputs[i+1:i+4])
                   for i in range(len(inputs) - 3))

In [55]:
deeper2(sample1)

5

In [87]:
answer('sample1', deeper_quant2(sample1), 5)

True

In [56]:
deeper2(inp1[:10])

6

In [57]:
deeper2(inp1)

1611

In [88]:
answer('sample1', deeper_quant2(inp1), 1611)

True

## Day 2